<a href="https://colab.research.google.com/github/AsiyatShch/Internship/blob/master/Inference_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipympl
!pip install catboost
!pip install serial

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import os
import warnings
import pickle
import time
import serial
import matplotlib.pyplot as plt
from IPython.display import clear_output
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing train dataset and modeling

In [5]:
# Target
def make_y(gestures):

  le = LabelEncoder()
  state = gestures['state'].values
  y_cmd = le.fit_transform(state)

  return y_cmd

In [6]:
# Preprocessing functions
def smooth(df):
  alpha = 0.1
  df = pd.DataFrame(df)
  X_smoothed = pd.DataFrame()
  for column in df.columns:
      smoothed_values = [df[column].iloc[0]]  # Initialize with the first observed value
      for i in range(1, len(df)):
          smoothed_value = alpha * df[column].iloc[i] + (1 - alpha) * smoothed_values[-1]
          smoothed_values.append(smoothed_value)
      X_smoothed[column] = smoothed_values
  return X_smoothed

def smooth_scale(x, scaler):
    df_smoothed = smooth(x)  # Pass the input variable 'x' instead of 'df_selected'
    array_scaled = scaler.fit_transform(x)  # Use 'x' for scaling
    df_scaled = pd.DataFrame(array_scaled)
    return df_scaled

def imp_features(df, y):
    OMG_CH = [i for i in range(50)]
    df['gest'] = pd.Series(y)
    df['label'] = df['gest'].apply(lambda x: '0' if x == 0 else '1')
    std_dev_per_label = df.groupby('label', as_index=False)[OMG_CH].std().T.reset_index().rename(columns={'index':'column'}).drop(0, axis=0)
    std_dev_per_label['std_dif'] = (std_dev_per_label[1]-std_dev_per_label[0]).abs()
    imp_features_list = list(std_dev_per_label.sort_values(by='std_dif', ascending=False)['column'][:12])
    df_selected = df[imp_features_list]
    imp_features_list = [str(el) for el in imp_features_list]
    df_selected.columns = df_selected.columns.astype(str)
    return df_selected,imp_features_list



In [7]:
# Train dataset
montage_train = '2024-03-04_08-17-49.emg8'

folder = '/content/drive/MyDrive/Моторика'
filepath_train = os.path.join(folder, montage_train)


gestures = pd.read_csv(filepath_train, sep=' ', on_bad_lines='warn')
gestures = gestures[~gestures['state'].isin(['Baseline', 'Finish'])]
gestures.sort_index(inplace=True)
gestures.reset_index(drop=True, inplace=True)

OMG_CH = np.arange(50).astype(str)
X = gestures[OMG_CH]
display(X.columns)

le = LabelEncoder()
state = gestures['state'].values
y = le.fit_transform(state)


scaler = StandardScaler()
X = smooth_scale(X, scaler)
X,imp_features_list = imp_features(X, y)

display(X.shape, y.shape,
        imp_features_list)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49'],
      dtype='object')

(6364, 12)

(6364,)

['16', '36', '46', '6', '26', '13', '3', '23', '33', '43', '11', '1']

In [8]:
# Creating a model
model_boost = CatBoostClassifier(depth=7, eval_metric='Accuracy', verbose=False, iterations=30)

model_boost.fit(X, y)

# Preprocessing new data and inference

In [9]:
def preprocessing(x):
  x = smooth_scale(x, scaler)
  x.columns = x.columns.astype(str)
  x = x[imp_features_list]
  return x

In [10]:
def inference(x):
    y = model_boost.predict(x)
    return y

In [11]:
def postprocessing(x, prev):
    if prev is None:
        y = x
    else:
        y = x*0.1 + prev*0.9 # Holt-Winters filter
    return y

def commands(x):
    y = np.round(np.clip(x / 100, 0, 1)*100).astype(int)
    return y

def commands(x):
    cmd = vp.neutral_state.copy()
    # for this mapping look at GESTURES
    if x == 0: # 'Close'
        cmd.update({'thumb_flex': 1, 'index_flex': 1, 'middle_flex': 1, 'ring_flex': 1, 'pinky_flex': 1})
        return cmd
    elif x == 1: # 'Indication'
        cmd.update({'thumb_extend':1, 'index_extend': 1, 'middle_flex': 1, 'ring_flex': 1, 'pinky_flex': 1})
        return cmd
    elif x == 2: # 'Neutral'
        pass
        return cmd
    elif x == 3: # 'Open'
        cmd.update({'thumb_extend': 1, 'index_extend': 1, 'middle_extend': 1, 'ring_extend': 1, 'pinky_extend': 1})
        return cmd
    elif x == 4: # 'Pinch'
        cmd.update({'thumb_flex':1, 'index_flex': 1, 'middle_extend': 1, 'ring_extend': 1, 'pinky_extend': 1})
        return cmd
    elif x == 5: # 'ThumbFingers'
        cmd.update({'thumb_flex': 1, 'index_extend': 1, 'middle_extend': 1, 'ring_extend': 1, 'pinky_extend': 1})
        return cmd
    else:
        return cmd

In [12]:
montage_test = '2024-03-04_10-00-21.emg8'

import os
import warnings

folder = '/content/drive/MyDrive/motorika'
filepath_train = os.path.join(folder, montage_train)

gestures = pd.read_csv(montage_test, sep=' ', on_bad_lines='warn')
gestures = gestures[~gestures['state'].isin(['Baseline', 'Finish'])]
gestures.sort_index(inplace=True)
gestures.reset_index(drop=True, inplace=True)

OMG_CH = np.arange(50).astype(str)
X_test = gestures[OMG_CH]
display(X_test.columns)


Skipping line 3: expected 56 fields, saw 74



Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49'],
      dtype='object')

In [13]:
le = LabelEncoder()
state = gestures['state'].values
y_true = le.fit_transform(state)

In [14]:
X_test = smooth_scale(X_test, scaler)
X_test.columns = X_test.columns.astype(str)

X_test = X_test[imp_features_list]
X_test.columns

Index(['16', '36', '46', '6', '26', '13', '3', '23', '33', '43', '11', '1'], dtype='object')

In [ ]:
TIMEOUT = 0.033
DEBUG = False

i = 0
ts_old = time.time()
ts_diff = 0;

y_previous = None
y_dct = {
    'omg_sample':[],
    'sample_preprocessed':[],

    'y_predicted':[],
    'y_postprocessed':[],
    'y_commands':[],
}
while True:

    # [Data reading]
    ts_start = time.time()

    try:
        # [Sim data]
        if i < len(X_test):
            sample = X_test.values[i]
        else:
            break
        # [/Sim data]
        [[ts, cycle_ts], omg_sample] = np.array_split(sample, [2])

    except Exception as e:
        print(e)

    # [/Data Reading]

    # [Data preprocessing]
    omg_sample = pd.DataFrame(omg_sample)
    sample_preprocessed = preprocessing(omg_sample.values.reshape(1, -1))
    # [/Data preprocessing]

    # [Inference]
    y_predicted         = inference(sample_preprocessed)
    # [/Inference]

    # [Inference Postprocessing]
    y_postprocessed     = postprocessing(y_predicted, y_previous)
    # [/Inference Postprocessing]

    # [Commands composition]
    y_commands          = commands(y_postprocessed.argmax())
    # [/Commands composition]

    # [Commands sending]
    # NO COMMANDS SENDING IN SIMULATION
    # [/Commands sending]

    # [Data logging]
    y_dct['omg_sample'].append(omg_sample)
    y_dct['sample_preprocessed'].append(sample_preprocessed)
    y_dct['y_predicted'].append(y_predicted)
    y_dct['y_postprocessed'].append(y_postprocessed)
    y_dct['y_commands'].append(y_commands)
    # [/Data logging]

    y_previous = y_postprocessed

    if DEBUG:
        clear_output(wait=True)

        # sanity check: Sizes of SAMPLE=52, OMG=50
        print(f'SAMPLE SIZE: {len(sample)}, OMG: {len(omg_sample)}')
        # print('INPUT:\n', s)
        print('SAMPLE:\n', sample)

        # sanity check: Sizes of SAMPLE=65, OMG=50
        print(f'SAMPLE SIZE: {len(sample)}, OMG: {len(omg_sample)}')
        print(f'TS: {ts}, CYCLE TS: {cycle_ts}')
        print(y_postprocessed)
        print(y_commands)

    ts_diff = time.time() - ts_start
    assert(ts_diff<TIMEOUT), 'Calculation cycle takes more than TIMEOUT, halting...'
    ts_old = ts_start
    i += 1

NameError: name 'vp' is not defined

In [ ]:
for key, val in y_dct.items():
    # print(f"len({key}) = {len(y_dct[key])}")
    y_dct[key] = np.stack(val)
    print(f"{key}.shape = {y_dct[key].shape}")

In [ ]:
from copy import deepcopy
import MotoricaInterface as mi; assert(mi.__version__=="0.0.3") # last time tested
vp = mi.VirtualPhantomClient('localhost', 1755)

In [ ]:
!ls /dev/ttyUSB*

In [ ]:
TIMEOUT = 0.033
DEBUG = True

ser = None
ser_port = None
ser_port = '/dev/ttyUSB1'

if ser_port is not None:
    ser = serial.Serial(port=ser_port, baudrate=115200, timeout=1.0)
    # time.sleep(0.2)

    # ser.write('M0\r\n'.encode())
    # time.sleep(0.1)

    # # ser.write('S2\r\n'.encode())
    # # time.sleep(0.1)

    # ser.write('PT\r\n'.encode())
    # time.sleep(0.1)
    # ser.write('Pt\r\n'.encode())
    # time.sleep(0.1)
    # ser.flush()

    # time.sleep(0.1)

In [ ]:
def parseline():
    line = ser.readline().decode().replace('\n', '')
    data = np.array(list(filter(len, np.array(line.split(" "))))).astype(float)
    return data

ser.readline()
while True:
    data = parseline()
    print(f"data len: {len(data)} |", list(data), end = '                                          \r')

In [ ]:
# flush buffers
ser.reset_input_buffer()
ser.read()

i = 0;
while(ser.in_waiting):
    print(f'Flushing buffers {i}: {ser.in_waiting}', end='    \r')
    ser.read_all()
    time.sleep(0.005)
    i+=1;
ser.readline()
ser.readline()

i = 0
ts_old = time.time()
ts_diff = 0;

ts_diffs = []
x_previous = None
y_previous = None

pack = None

prev_state = 'None'
prev_idx = -1
y_commands = vp.neutral_state.copy()

while True:

    # [Data reading]
    s = ser.readline()
    ts_start = time.time()

    try:
        sample = s.decode().replace('\n', '')
        data = np.array(list(filter(len, np.array(sample.split(" "))))).astype(float)
        [[ts, cycle_ts], omg_sample] = np.array_split(data, [2])

    except Exception as e:
        print(e)
    # [/Data Reading]

    # [Data preprocessing]
    sample_preprocessed = preprocessing(omg_sample)
    # [/Data preprocessing]

    # [Inference]
    y_predicted         = inference(sample_preprocessed)
    # [/Inference]

    # [Inference Postprocessing]
    y_postprocessed     = postprocessing(y_predicted, y_previous)
    # [/Inference Postprocessing]

    # [Commands composition]
    y_commands          = commands(y_postprocessed.argmax())
    # [/Commands composition]

    # [Commands sending]
    # pack, _ = drv_abs_one(ser, list(y_commands)); # + [0]
    prot_idx = deepcopy(y_commands)
    if (prot_idx!=prev_idx):
        # socket_send(row.Command)
        vp.send_cmd(**y_commands)
        prev_idx = prot_idx
    # [/Commands sending]

    y_previous = y_postprocessed


    if DEBUG:
        clear_output(wait=True)

        # sanity check: iteration should increase monotonically, TIMEDIFF approximately 32-34 ms, CYCLETIME < TIMEOUT, WAITING should be == 0
        print(f'ITERATION:\t{i}\tTIMEDIFF:\t{(ts_start - ts_old)*1000: .0f}\tCYCLETIME:\t{ts_diff*1000:.0f}\tWAITING:\t{ser.in_waiting}')
        # print('INPUT:\n', s)
        print('SAMPLE:\n', sample)

        # sanity check: Sizes of SAMPLE=65, OMG=50
        print(f'SAMPLE SIZE: {len(sample)}, OMG: {len(omg_sample)}')
        print(f'TS: {ts}, CYCLE TS: {cycle_ts}')
        print(y_postprocessed, GESTURES[y_postprocessed])
        # print(y_commands)
        # print(pack)

    ts_diff = time.time() - ts_start
    # assert(ts_diff<TIMEOUT), 'Calculation cycle takes more than TIMEOUT, halting...'
    ts_old = ts_start
    i += 1